# Реализация классификатора $KFDA$ (Kernel Fisher Discriminant Analysis):
(Примеры использования и анализ см. в других файлах)

In [20]:
# Необходимые модули:
from sklearn.base import BaseEstimator, ClassifierMixin
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import scipy
from scipy.spatial import distance
import math
from  scipy import stats, linalg, exp
from sklearn.neighbors import KNeighborsClassifier  
import nbimporter
from numpy import linalg
from sklearn.svm import SVC
import scipy.io as sio

In [27]:
from sklearn.base import BaseEstimator, ClassifierMixin
class KFDACl(BaseEstimator, ClassifierMixin): #Класс, для реализации KFDA
    def __init__(self, kernel = 'RBF', gamma = 'auto', c= None, d = None, Alg = 'SVM'):
        """    
               parameters:
               kernel - string, тип ядра: linear, Polynomial, RBF
               gamma - float, параметр RBF kernel. Если gamma = 'auto', то gamma = '1/n', где n - размерность пространства)
               c, d - int,  параметры Polynomial kernel(cвободный член и степень соответсвенно)
               Alg - string, способ классификации после понижения размерности: KNN, SVM
        """
        self.kernel = kernel
        self.gamma = gamma
        self.c = c
        self.d = d
        self.Alg = Alg

    def fit(self, X, y):
        """
        input:
               X - np.ndarray, матрица признаков объектов  nxk 
               y - np.array, вектор меток классов nx1 
        """
        if (self.gamma =='auto'):
            self.gamma = 1/(X.shape[1])
        self.labels_ = np.unique(y) # находим значение меток классов
        self.Cl_ = len(self.labels_) #Кол-во классов
        self.cl_ = [np.nonzero(y == self.labels_[i])[0] for i in range(0,self.Cl_)]
        Cl =self.Cl_
        K = self.Kernelmtrx(X, X) 
        #Разделяем образцы по классам
        Xcl = [X[self.cl_[i],:] for i in range(0,Cl)]
        #Кол-во образцов в каждом из классов
        ni = [Xcl[i].shape[0] for i in range (0,Cl)]
        n= sum(ni)
        #Вычисляем kernel matrix
        Ki = [K[:,np.nonzero(y == self.labels_[i])[0]] for i in range(0,Cl)]
        #Вычисляем Mi
        Mi = [np.mean(Ki[i], axis= 1) for i in range (0,Cl)]
        M0 = np.array([np.mean(K, axis=1)])
        M = np.zeros((n,n))
        for i in range (0,Cl):
            M = M + np.dot((M0-Mi[i]).T, M0-Mi[i])
        #Вычисляем Ni
        I = [np.eye(ni[i]) for i in range (0,Cl)]
        O = [1/float(ni[i]) for i in range (0,Cl)]
        T = [(I[i] - O[i]) for i in range (0, Cl)]
        Ni = [np.dot(Ki[i], np.dot(T[i], Ki[i].T)) for i in range (0, Cl)]
        #Добавляем mu (для регуляризации)
        eps=np.eye((n))
        eps = eps * 0.001
        N= sum(Ni) + eps
        e, v = np.linalg.eigh(M) #находим собственные значения и вектора M
        eps = np.ones(len(e))*abs(min(e))
        e = e + 2*eps
        sqrtE = [math.sqrt(x) for x in e]
        sqrtM =  np.dot(np.dot(v,np.diag(sqrtE)), np.matrix.transpose(v))
        S = np.dot(sqrtM, np.dot(np.linalg.inv(N), sqrtM))
        eigenValues, eigenVectors = scipy.linalg.eigh(S)
        self.alpha_ = np.zeros((eigenVectors.shape[0], Cl-1)) #alpha, вектор/матрица для проекции данных в расширенном простр-ве
        idx = eigenValues.argsort()[::-1] 
        eigenValues = eigenValues[idx]
        eigenVectors = eigenVectors[:,idx]
        #alpha матрица для проекции, состоящая из Сl-1 собств векторов, соотв. Сl-1 наиб. собств. значениям матрицы S:
        for i in range (0, Cl):
            self.alpha_[:,i-1] =  np.dot(np.linalg.inv(sqrtM),eigenVectors[:, Cl-i-1])
        Z=np.dot(K,self.alpha_)
        # Значение образцов в проекции в Сl-1 пространство:
        self.X_red_ = Z.reshape(Z.shape[0],(Cl-1))
        self.Xtr_= X 
        # После понижения размерности, для спроектированных данных строим предсказательную модель:
        if (self.Alg == 'KNN'):
                
                PrModel = KNeighborsClassifier(n_neighbors=5)  
                PrModel.fit(self.X_red_, y)  
                self.PrModel_=PrModel
                print (PrModel.score(self.X_red_,y))         
                
        if (self.Alg == 'SVM'):
            if (self.Cl_ == 2): 
                PrModel= SVC(kernel = 'linear', probability =True)
            else:
                PrModel= SVC(probability =True)
            PrModel.fit(self.X_red_, y)
            self.PrModel_=PrModel
        return self
    def predict(self,Xtst):
        """
        input: Xtst - np.ndarray, матрица признаков объектов, для которых необх предсказать класс, mxk
        output:
               ypred - вектор предсказанный меток классов для объектов из Xpr, mx1
        """
        Cl = self.Cl_
        Kpr = self.Kernelmtrx(self.Xtr_, Xtst) 
        Xtst_red = np.dot(Kpr, self.alpha_) #Проектируем данные в уже найденное (в ф-ции fit) (Cl-1) мерное подпространство
        Xtst_red = Xtst_red.reshape(Xtst_red.shape[0],(Cl-1))
        ypred = np.zeros((Xtst_red.shape[0],1))
        #После проекции вектора в (Cl-1) мерное пространство определяем его к классу по построенной (в ф-ции fit)  предс модели:
        y_pred = self.PrModel_.predict(Xtst_red)  
        return y_pred
    def predict_proba(self, Xtst):
        Cl = self.Cl_
        Kpr = self.Kernelmtrx(self.Xtr_, Xtst) 
        w = self.alpha_
        Xtst_red = np.dot(Kpr, w) 
        Xtst_red = Xtst_red.reshape(Xtst_red.shape[0],(Cl-1)) 
        return self.PrModel_.predict_proba(Xtst_red)
    def decision_function(self, Xtst):
        Cl = self.Cl_
        Kpr = self.Kernelmtrx(self.Xtr_, Xtst) 
        w = self.alpha_
        Xtst_red = np.dot(Kpr, w) 
        Xtst_red = Xtst_red.reshape(Xtst_red.shape[0],(Cl-1)) 
        return self.PrModel_.decision_function(Xtst_red)
    def Kernelmtrx(self, Xtr, Xtst): #расчитывает kernel матрицу для векторов признаков 1ой группы объектов и 2ой, mxn
        """
        input: Xtr - матрица признаков 1ой группы объектов nxk
               Xtst - матрица признаков другой группы объектов mxk
        """ 
        K = np.zeros((Xtst.shape[0], Xtr.shape[0]))
        if (self.kernel == 'linear'):   #В случае linear kernel KFDA будет равносильно FDA
            return np.dot(Xtst, Xtr.T)
        if (self.kernel == 'RBF'):   #RBF kernel
            D = scipy.spatial.distance.cdist(Xtst, Xtr, 'sqeuclidean')
            K=exp(-0.5*D/(self.gamma*self.gamma))
            return K
        if (self.kernel =='Polynomial'): #K(x,y) = ((x, y.T) + b)^d
                if (self.c == None): 
                    print ("Введите свободный параметр c Polynomial Kernel")
                    self.c = int(input(),10)
                if (self.d == None): 
                    print ("Введите степень d Polynomial Kernel")
                    self.d = int(input(),10)
                K = (np.dot(Xtst, Xtr.T)+ self.c)**(self.d)
                return K
    def transform(self, Xtst , Proj): 
        """
        input: Xtst - np.ndarray, матрица признаков объектов, для которых необх. предсказать класс, mxk
               Proj - int, размерность подпространства, в которое проецируем: (Cl-1)<=Proj <=k
        output: Xtst_red -  данные Xtst, спроецированные  в подпространство из Proj "наилучших" векторов, найденных КFDA(fit)
               
        """
        Cl = self.Cl_
        Kpr = self.Kernelmtrx(self.Xtr_, Xtst) 
        w = self.alpha_[:,:Proj]
        Xtst_red = np.dot(Kpr, w) 
        Xtst_red = Xtst_red.reshape(Xtst_red.shape[0],Proj)
        return (Xtst_red)
    def score(self, X, y, k=5, t = 1, gamma = 0):
        """
        input: Xtst - np.ndarray, матрица тестовых объектов, mxk
               y - истинные значения меток тестовых объектов
        output: accuracy_score - float, ср. точность, с которой классификатор предсказывает метки классов
               
        """
        return accuracy_score(y, self.predict(X))